In [1]:
import numpy as np
import pandas as pd

from torch.utils.data.dataset import Dataset
from torch.utils.data.dataloader import DataLoader

In [2]:
class DS(Dataset):

    def __init__(self, transform=None):
        self.transform = transform

    def __getitem__(self, index):
        return [np.random.randint(0, 10, size=(3, 10, 10)) for i in range(5)]

    def __len__(self):
        return 100

In [3]:
test_loader = DataLoader(DS(), batch_size=8, shuffle=False)

In [4]:
for a in test_loader:
    break

In [6]:
len(a)

5

In [9]:
a[1].shape

torch.Size([8, 3, 10, 10])